# Universidade Federal de Minas Gerais
## Computação Evolucionária - TTC
### Trabalho Prático 2 - Questão 1

Daniela Amaral Sampaio - 2017074351

Matheus Brito Faria - 2017074386

## 1. Introdução

A questão 1 do trabalho prático 2 tem como objetivo projetar um algoritmo genético que evolui uma população de strings até convergir para a
seguinte string:

`METHINKS*IT*IS*LIKE*A*WEASEL`

## 2. Implementação

In [ ]:
import string
import numpy as np
from functools import reduce

### 2.1. Probabilidade de ser igual à string alvo

*   Qual a probabilidade de uma string gerada aleatoriamente ser exatamente igual à string alvo?

Considerando que são 26 letras no alfabeto, 10 digitos e o caracter de espaço temos:

In [ ]:
size_alphabet = 26
size_digits = 10
size_space = 1
num_possibilities = size_alphabet + size_digits + size_space

target_string = 'METHINKS IT IS LIKE A WEASEL'
size_target = len(target_string)

hit_target_prob = reduce(lambda a, b: a*b, [1/num_possibilities]*size_target)

print(f'A probabilidade de gerar aleatoriamente a string {target_string}'\
      f' é de {hit_target_prob}')

A probabilidade de gerar aleatoriamente a string METHINKS IT IS LIKE A WEASEL é de 1.2312655436389027e-44


### 2.1. Métodos Implementados

Dois métodos foram implementados nesse trabalho:



1. Um deles utiliza mutação e cruzamento onde a cada iteração, dentro de uma certa probabilidade, são selecionados 40% dos individuos via wheel roullete para reproduzir e cada um deles gera mais dois individuos via particionamento, são removidos os 40% piores da geração e adicionado os novos indivíduos filhos.
Para a mutação foi implementado um bit flip onde, dentro de uma certa probabilidade, cada indivíduo pode sofrer uma mutação também aleatória.

```
solver = WeaselProblemSolver()
solver.fit()
```
2. No outro método é usado uma abordagem diferente, onde a cada iteração o melhor individual é replicado para toda a população e todos individuos identicos são submetidos a mutação de bit flip, com certa probabilidade, em todas as iterações. 

```
solver = WeaselProblemSolver()
solver.fit_replicate()
```

Na classe `WeaselProblemSolver` são possíveis observar várias funções, em especial, destacam-se:


*   `def calculate_fitness`: Função para apontar qual a string buscada e identificar o quão próxima a string gerada pelo código está próxima da string desejada.
*   `def calculate_spf`: Calcula-se o Operador de Seleção Proporcional ao Fitness (SPF), que é a probabilidade de seleção para o i-ésimo indivíduo. Faz-se a divisão entre o fitness do individuo atual e a soma de todos os fitness da população.
*   `def run_roulette_wheel`: Função utilizada para escolher indivíduos da geração atual para fazerem parte da geração futura, através de um sorteio que segue o seguinte método: cada indivíduo tem um 'valor' de aptidão, quanto maior a aptidão, maior a chance de ser escolhido na roleta. A roleta gira um determinado número de vezes dependendo do tamanho da população.
*  ` def run_crossover:` Função utilizada para realizar o crossover com probabilidade de 70%. Substitui os pais pelos filhos, removendo os piores indivíduos e inserindo novos na população.
* ` def run_mutation:` Função utilizada para gerar a mutação dos indivíduos com probabilidade de mutação de 15%. Chama a função `def apply_bit_flip` que faz uma troca aleatória dos bits.




In [ ]:
class WeaselProblemSolver:
    def __init__(self, 
                 target_string='METHINKS IT IS LIKE A WEASEL',
                 population_size=100,
                 crossover_rate=0.7,
                 mutation_rate=0.15,
                 bit_flip_rate=0.05):
        self.target_string = target_string.upper()
        self.target_size = len(target_string)
        self.create_encoder()
        self.target = self.encode(self.target_string)
        self.population_size = population_size
        self.crossover_rate = crossover_rate
        self.mutation_rate = mutation_rate
        self.bit_flip_rate = bit_flip_rate
        self.top_individuals = list()

    def create_encoder(self):
        self.encoder = {str(i): i for i in range(10)}
        self.encoder.update(dict(zip(string.ascii_uppercase, range(10, 37))))
        self.encoder.update({' ': 36})
        self.create_decoder()

    def create_decoder(self):
        self.decoder = {v: k for k, v in self.encoder.items()}

    def encode(self, string_to_encode):
        return np.array(list(map(lambda x: self.encoder[x], string_to_encode)))

    def decode(self, array):
        return "".join(map(lambda x: self.decoder[x], array))

    def initialize_population(self):
        self.population = np.random.randint(37, size=(self.population_size, 
                                                     self.target_size))
        
    def apply_bit_flip(self):
        for individual_index in range(self.population_size):
            for gene_in dex in range(self.target_size):
                if np.random.rand() < self.bit_flip_rate:
                    self.population[individual_index, gene_index] = \
                    np.random.randint(37)

    def run_mutation(self):
        self.apply_bit_flip()

    def print_population(self):
        for individual in self.population:
            print(self.decode(individual))
    
    def calculate_fitness(self):
        self.fitness = np.array([sum(individual == self.target) 
                                for individual in self.population])
        self.fitness_ranking = np.argsort(-1*self.fitness)
        return self.fitness

    def replicate_individual(self, individual):
        self.population = np.tile(
            individual, reps=(self.population_size, 1))
        return self.population
        
    def get_best_individual(self):
        best_one = np.argmax(self.fitness)
        self.top_individuals.append(self.population[best_one])
        return self.top_individuals[-1]

    def calculate_spf(self):
        self.spf = self.fitness / np.sum(self.fitness)
        return self.spf
    
    def run_roulette_wheel(self):
        self.calculate_spf()
        parents_index = list()
        probs = [sum(self.spf[:i+1]) for i in range(self.population_size)]
        for _ in range(self.population_size):
            random_value = np.random.rand()
            for (individual_index, individual_spf) in enumerate(probs):
                if random_value <= individual_spf:
                    parents_index.append(individual_index)
                    break
        self.parents_index = np.array(parents_index)
        return self.parents_index

    def select_parents(self):
        self.number_of_parents = int(self.population_size * 0.4)
        self.number_of_parents = self.number_of_parents \
        if self.number_of_parents % 2 == 0 else self.number_of_parents + 1
        self.run_roulette_wheel()
        selection = self.population[self.parents_index]
        pairs = [[selection[i], selection[i+1]] 
                 for i in range(0, self.number_of_parents, 2)]
        parents = np.array(pairs)
        return parents

    def run_cut_and_crossfill(self, parents):
        childrens = list()
        for pair in parents:
            cut_point = np.random.randint(1, self.target_size)
            save_slice = pair[1][:cut_point].copy()
            pair[1][:cut_point] = pair[0][:cut_point]
            pair[0][:cut_point] = save_slice
            childrens.append(pair[0])
            childrens.append(pair[1])
        return np.array(childrens)

    def remove_worse_individuals(self):
        self.population = np.delete(
            self.population, 
            self.fitness_ranking[::-1][:self.number_of_parents], 
            axis=0)
        
    def insert_childrens_in_population(self, childrens):
        self.population = np.append(
            self.population, 
            childrens, 
            axis=0)

    def run_crossover(self):
        parents = self.select_parents()
        childrens = self.run_cut_and_crossfill(parents)
        self.remove_worse_individuals()
        self.insert_childrens_in_population(childrens)

    def fit_replicate(self, max_iterations=1_000, _print=True):
        self.initialize_population()
        self.calculate_fitness()
        best_individual = self.get_best_individual()

        list_fitness = list()
        iteration = 1
        while iteration <= max_iterations and (best_individual != self.target).any():
            self.replicate_individual(best_individual)
            self.run_mutation()
            self.calculate_fitness()
            list_fitness.append(max(self.fitness))
            best_individual = self.get_best_individual()
            if _print:
                print(f'Iteration {iteration}: {self.decode(best_individual)}')
            iteration += 1
        return list_fitness

    def fit(self, max_iterations=1_000, _print=True):
        self.initialize_population()
        self.calculate_fitness()
        best_individual = self.get_best_individual()

        list_fitness = list()
        iteration = 1
        while iteration <= max_iterations and (best_individual != self.target).any():
            if np.random.rand() < self.crossover_rate:
                self.run_crossover()
            if np.random.rand() < self.mutation_rate:
                self.run_mutation()
            self.calculate_fitness()
            list_fitness.append(max(self.fitness))
            best_individual = self.get_best_individual()
            if _print:
                print(f'Iteration {iteration}: {self.decode(best_individual)}')
            iteration += 1
        return list_fitness

Usando a abordagem ```.fit()``` com 100 indivíduos em cada geração, um ```crossover_rate``` de 0.7, ```mutation_rate``` de 0.15 e ```bit_flip_rate``` de 0.05.


In [ ]:
solver = WeaselProblemSolver()
_ = solver.fit()

Iteration 1: 5N1HR FX07VTOSLLGILLV8E5YP1L
Iteration 2: 5N1HR FX07VTOSLLGILLV8E5YP1L
Iteration 3: 5N1HR FX07VTOSLLGILLV8E5YP1L
Iteration 4: 5N1HR FX07VTOSLLGILLV8E5YP1L
Iteration 5: O5I0U5YCW0TB7S E5CEFA7IFGL2F
Iteration 6: O5I0U5YCW0TB7S E5CEFA7IFGL2F
Iteration 7: O5I0U5YCW0TB7S E5CEFA7IFGL2F
Iteration 8: O5I0U5YCW0TB7S E5CEFA7IFGL2F
Iteration 9: MC56IDS3FRGB7S E5CEFA7IFGL2F
Iteration 10: MC56IDS3FRGB7S E5CEFA7IFGL2F
Iteration 11: MC56IDS3FRGB7S E5CEFA7IFGL2F
Iteration 12: MC56IHIP8ITA6G9 VQ9 O89DAUAL
Iteration 13: MC56IHIP8ITA6G9 VQ9 O89DAUAL
Iteration 14: MC56IHIP8ITA6G9 VQ9 O89DAUAL
Iteration 15: MC56IHIP8ITA6G9 VQ9 O89DAUAL
Iteration 16: MC56IHIP8ITA6G9 VQ9 O89DAUAL
Iteration 17: MC56IHIP8ITA6G9 VQ9 O89DAUAL
Iteration 18: MC56IHIP8ITQ6G9 VCQ O89DAUAL
Iteration 19: MC56IHIP8ITQ6G9 VCQ O89DAUAL
Iteration 20: MC56IHIP8ITQ6G9 VCQ O89DAUAL
Iteration 21: MC56IHIP8ITAGA3ZRKEKA7XB581L
Iteration 22: MC5AIDS3WI0TOSLLT8E O89DAUAL
Iteration 23: MC5AIDS3WI0TOSLLT8E O89DAUAL
Iteration 24: MC5AID

Verificando o número médio de gerações para convergência para a abordagem ```.fit()```

In [ ]:
num_iteration_list = list()
for _ in range(5):
    solver = WeaselProblemSolver()
    num_iteration_list.append(len(solver.fit(_print=False)))

print(f'O numero medio de iteracoes foi de {np.mean(num_iteration_list)}')

O numero medio de iteracoes foi de 468.8


Usando a abordagem ```.fit()``` com 100 indivíduos em cada geração, ```mutation_rate``` de 0.15,  ```bit_flip_rate``` de 0.05 e variando o valor de ```crossover_rate```.

In [ ]:
rate_list = np.linspace(0, 1, 6)
for crossover_rate in rate_list:
    num_iteration_list = list()
    for _ in range(5):
        solver = WeaselProblemSolver(crossover_rate=crossover_rate)
        num_iteration_list.append(len(solver.fit(_print=False)))
    print(f'Usando o crossover_rate={round(crossover_rate, 1)} foram '\
          f'gastas em media {round(np.mean(num_iteration_list), 1)} iteracoes.')

Usando o crossover_rate=0.0 foram gastas em media 1000.0 iteracoes.
Usando o crossover_rate=0.2 foram gastas em media 1000.0 iteracoes.
Usando o crossover_rate=0.4 foram gastas em media 831.0 iteracoes.
Usando o crossover_rate=0.6 foram gastas em media 620.8 iteracoes.
Usando o crossover_rate=0.8 foram gastas em media 533.4 iteracoes.
Usando o crossover_rate=1.0 foram gastas em media 443.8 iteracoes.


Usando a abordagem ```.fit()``` com 100 indivíduos em cada geração, ```crossover_rate```  de 0.7,  ```bit_flip_rate``` de 0.05 e variando o valor de ```mutation_rate```.

In [ ]:
rate_list = np.linspace(0, 1, 6)
for mutation_rate in rate_list:
    num_iteration_list = list()
    for _ in range(5):
        solver = WeaselProblemSolver(mutation_rate=mutation_rate)
        num_iteration_list.append(len(solver.fit(_print=False)))
    print(f'Usando o mutation_rate={round(mutation_rate, 1)} foram '\
          f'gastas em media {round(np.mean(num_iteration_list), 1)} iteracoes.')

Usando o mutation_rate=0.0 foram gastas em media 1000.0 iteracoes.
Usando o mutation_rate=0.2 foram gastas em media 424.0 iteracoes.
Usando o mutation_rate=0.4 foram gastas em media 1000.0 iteracoes.
Usando o mutation_rate=0.6 foram gastas em media 1000.0 iteracoes.
Usando o mutation_rate=0.8 foram gastas em media 1000.0 iteracoes.
Usando o mutation_rate=1.0 foram gastas em media 1000.0 iteracoes.


Fazendo uma análise mais restrita nesse parâmetro.

In [ ]:
rate_list = np.linspace(0.1, 0.3, 6)
for mutation_rate in rate_list:
    num_iteration_list = list()
    for _ in range(5):
        solver = WeaselProblemSolver(mutation_rate=mutation_rate)
        num_iteration_list.append(len(solver.fit(_print=False)))

    print(f'Usando o mutation_rate={round(mutation_rate, 1)} foram '\
          f'gastas em media {round(np.mean(num_iteration_list), 1)} iteracoes.')

Usando o mutation_rate=0.1 foram gastas em media 481.8 iteracoes.
Usando o mutation_rate=0.1 foram gastas em media 424.8 iteracoes.
Usando o mutation_rate=0.2 foram gastas em media 417.0 iteracoes.
Usando o mutation_rate=0.2 foram gastas em media 461.0 iteracoes.
Usando o mutation_rate=0.3 foram gastas em media 417.0 iteracoes.
Usando o mutation_rate=0.3 foram gastas em media 542.4 iteracoes.


Usando a abordagem ```.fit_replicate()``` com 100 indivíduos em cada geração e  ```bit_flip_rate``` de 0.05.



In [ ]:
solver = WeaselProblemSolver()
_ = solver.fit_replicate()

Iteration 1: CTPQ4NOC9L TQTVDOKWD WWEK 7V
Iteration 2: DTPQ4NMC9L TQTVDOKWD WWEK SV
Iteration 3: DTPQINMC9L TQTVDOKWD WWEK SV
Iteration 4: DEPQINMR9L TQTVDOKCD WWEK SV
Iteration 5: DEPQINMR9L TQTVDIKCD WWEK SV
Iteration 6: DEPQINMR9LTTTTXDIKCD WWEK SV
Iteration 7: DETQINMR9LTTTPXDIKCH WWEK SV
Iteration 8: DETQINMR9LTTTPXDIKCH WWEK SV
Iteration 9: DETQINMR9LTTIPXDIKCHOWWEK SV
Iteration 10: DETQINMR9LTKIPXDIKCHOWWEKSSV
Iteration 11: METQINMR9LTDIPXDIKCHOWWEKSSV
Iteration 12: METQINMR9LTDIP7DIKEHOWWEZSSV
Iteration 13: METQINMR9ITDIP7DIKEHOWWEZSS7
Iteration 14: METQINMR9ITDIP7DIKE OWWEZSS7
Iteration 15: METQINMR9ITDIP7DIKE OWWEZSS7
Iteration 16: METQINMR9ITDIP7DIKE OWWEZSS7
Iteration 17: METQINMS9ITDIP7DIKE OWWEZSS7
Iteration 18: METQINMS9IT IP7DIKE OWWEZSS7
Iteration 19: METQINMS9IT IP7DIKE OWWEZSS7
Iteration 20: METQINMS9IT IP7DIKE TWWEZSS7
Iteration 21: METQINMS9IT IP7DIKE BWWEZSS7
Iteration 22: METQINMS9IT IP7DIKE AWWEZSS7
Iteration 23: METQINMS9IT IP7DIKE AWWEZSS7
Iteration 24: METQIN

Verificando o número médio de gerações para convergência para a abordagem ```.fit_replicate```

In [ ]:
num_iteration_list = list()
for _ in range(5):
    solver = WeaselProblemSolver()
    num_iteration_list.append(len(solver.fit_replicate(_print=False)))

print(f'O numero medio de iteracoes foi de {np.mean(num_iteration_list)}')

O numero medio de iteracoes foi de 98.0


## 3. Resultados

* **Calcule a dimensão do espaço de busca para esse problema.
Qual a probabilidade de uma string gerada aleatoriamente ser exatamente igual à
string alvo?**

Considerando que são 26 letras no alfabeto, 10 digitos e o caracter de espaço temos que a probabilidade de gerar aleatoriamente a string METHINKS IT IS LIKE A WEASEL é de 1.2312655436389027e-44.


* **Explique como o algoritmo genético consegue superar essa probabilidade incrivelmente pequena.**

O algoritmo converge para a melhor solução a medida que vai encontrando bons indivíduos e que os cruzamentos e mutações gerem indivíduos melhores. Dessa forma, ele é capaz de superar a probabilidade e encontrar uma solução ótima.

* **Calcule o número médio de gerações para convergência.**

Para a abordagem `.fit()` o número médio de iteracões para convergir foi de 468.8.

Para a abordagem `.fit_replicate` o número médio de iteracões para convergir foi de 98.0.

* **Faça experimentos com o parâmetro probabilidade de mutação. O que acontece com o desempenho do algoritmo?**

Variando o rate é possível observar que usar um valor muito baixo não permite que o algoritmo sofra nenhuma mudança significativa, e usar um valor muito alto faz com que o não ocorra de fato uma convergência. Fazendo um ajuste fino é possível notar que os valores ótimo do parametro ficam entre 0.1 e 0.3.

* **Faça experimentos com o parâmetro probabilidade de cruzamento. O que acontece com o desempenho do algoritmo?**

Quanto maior o número de indivíduos a serem cruzados, mais rápido o algoritmo converge para a solução ótima.





## 4. Conclusão

Durante a execução do trabalho foi possível utilizar na prática conceitos aprendidos em sala de aula e acredita-se que o resultado final do trabalho tenha sido satisfatório, visto que conseguiu-se convergir para a string alvo nas duas abordagens utilizadas.